In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
plt.show()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/bigmart-sales-data/Train.csv')
test = pd.read_csv('../input/bigmart-sales-data/Test.csv')


In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.dtypes

# EDA

In [ ]:
train.hist(bins=50,figsize=(20,15)) 
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(train)

In [ ]:
 pd.crosstab(train.Outlet_Identifier, train.Item_Weight)



In [ ]:
 pd.crosstab(train.Outlet_Establishment_Year, train.Item_Outlet_Sales)



In [ ]:
train.groupby(["Outlet_Establishment_Year"])["Item_Outlet_Sales"].agg("mean").sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(212)

sns.boxplot(x ='Outlet_Establishment_Year' ,y='Item_Outlet_Sales',data=train,palette
="Set1" )

In [ ]:
ax = sns.violinplot(x="Outlet_Size", y="Item_Outlet_Sales", data=train)

In [ ]:
train.groupby(["Item_Type","Item_Fat_Content"])["Item_Fat_Content"].agg("count").sort_values().groupby("Item_Type").tail(1)

# Correalation

In [ ]:
corr_matrix=train.corr()
corr_matrix["Item_Outlet_Sales"].sort_values(ascending=False) 

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(train.corr(), annot=True)

# Data prepration

In [ ]:
bd = pd.concat([train,test],axis=0)

In [ ]:
bd.isnull().sum()

In [ ]:
bd['Item_Weight'].fillna( method ='ffill', inplace = True) 
  

In [ ]:
bd["Outlet_Size"].fillna( method ='ffill', inplace = True) 
  


In [ ]:
bd.isnull().sum()

In [ ]:
bd['Outlet_Size'].value_counts()

In [ ]:

bd['Outlet_Size']=bd['Outlet_Size'].replace({'Small':1,
                                             'Medium':2,
                                             'High':3})

In [ ]:
#correcting mispelled column
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace({'LF': 'lf',
                                                         'Low Fat':'lf',
                                                         'low fat':'lf',
                                                         'reg':'reg',
                                                         'Regular':'reg'
                                                        })

In [ ]:
items = bd["Item_Identifier"]
bd = bd.drop(columns="Item_Identifier")
bd = bd.drop(columns="Outlet_Identifier")
bd["Outlet_Year"] = 2020- bd["Outlet_Establishment_Year"]
bd =  bd.drop(columns="Outlet_Establishment_Year")

In [ ]:

bd.columns[bd.dtypes=='object']
bd = pd.get_dummies(bd,columns = bd.columns[bd.dtypes=='object'])

In [ ]:
train = bd[0:len(train)]
test = bd[len(train):]
test.drop(columns="Item_Outlet_Sales", inplace=True)

# train test split

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
X = train.drop(columns="Item_Outlet_Sales")
y = train["Item_Outlet_Sales"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state=5)


# ML Model

In [ ]:

sn = RandomForestRegressor(n_estimators=1200, max_depth=5,min_samples_split=2,min_samples_leaf=2)
sn.fit(X_train,y_train)
pred = sn.predict(X_test)
np.sqrt(mean_squared_error(y_test, pred))